### 행정안전부 도로명주소 API활용
- 체험하기 https://business.juso.go.kr/addrlink/openApi/apiExprn.do
- 여기에서 api신청해서 apikey받음 / 받아서 txt파일로 저장함
- key는 유출되지 않도록 gitignore에 저장해놓고 메일로 보관해둠
- 도로명주소를 갖고 카카오가서 위도경도를 찾고 dataframe에 얹어줘야함

In [1]:
import requests, json
from urllib.parse import quote      # 한글 인코딩

- API Key 가져오기

In [2]:
with open('data/roadapikey.txt') as f:
    road_key = f.read()

In [ ]:
# road_key 는 confmKey

- URL 만들기

In [ ]:
'''
엔드포인트(endpoint) 결정하기 # 꼭 API 제공 업체의 명세서를 복사해서 사용 직접입력X
API의 엔드포인트는 API를 호출할 때 사용되는 URL
파라미터(parameter) 결정하기(API 호출 시 전달할 데이터) /파라미터는 요청변수
엔드포인트 뒤에 "?" 기호를 추가 즉, "?"는 파라미터 시작
key=value 형태로 전달
여러 개의 파라미터를 전달해야 할 경우, "&" 기호로 구분 (파라미터와 파라미터 연결은 &)
한글은 인코딩해서 보내줘야함 %20%  => from urllib.parse import quote
'''

In [ ]:
'''
API 는 아래와 같은 http 코드로 응답

정상적인 응답에는 http 201 값을 반환

200 - 서비스 정상 제공중
201 - 요청 성공
400 - 잘못된 요청
403 - 접근 불가능
404 - 존재하지 않는 리소스
405 - 잘못된 요청 메소드
500 - 서버 오류. 신고 및 문의 게시판에 제보해주세요.
'''

In [4]:
# 한글은 반드시 인코딩해서 보내야 함
quote('경기도청')

'%EA%B2%BD%EA%B8%B0%EB%8F%84%EC%B2%AD'

In [5]:
# url 만들기
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do' #가이드에있는 API기본정보 get호출
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10' # currentPage, countPerPage 는 기본값이고 road_key도 고정값(승인받은 apikey)
params2 = f"keyword={quote('경기도청')}&resultType=json" # 인코딩된 키 넣어주기 
url = f'{base_url}?{params1}&{params2}'
# params2는 요청변수가 달라짐 원하는keyword가 달라질테니/인코딩해서 넣는 부분

- Open API를 호출하여 결과 받기

In [6]:
# 200 출력되면 잘된것
result = requests.get(url)
result.status_code # 코드상태확인

200

In [7]:
# ({}) JSON 
result.text[:200] # data 출력확인

'({"results":{"common":{"errorMessage":"정상","countPerPage":"10","totalCount":"68","errorCode":"0","currentPage":"1"},"juso":[{"detBdNmList":"","engAddr":"30 Docheong-ro, Yeongtong-gu, Suwon-si, Gyeongg'

- JSON data를 python에서 읽을 수 있도록 변환

In [8]:
# res = json.loads(result.text) #괄호 땜에 에러남 괄호없애기
# 시작과 끝에 ( )가 있어, 이것을 없애주어야 함
res = json.loads(result.text[1:-1])

- 딕셔너리 데이터에서 필요한 정보 추출하기

In [10]:
# keys확인
res.keys()

dict_keys(['results'])

In [12]:
res['results'].keys()

dict_keys(['common', 'juso'])

In [16]:
res['results']['juso']

[{'detBdNmList': '',
  'engAddr': '30 Docheong-ro, Yeongtong-gu, Suwon-si, Gyeonggi-do',
  'rn': '도청로',
  'emdNm': '이의동',
  'zipNo': '16508',
  'roadAddrPart2': ' (이의동)',
  'emdNo': '01',
  'sggNm': '수원시 영통구',
  'jibunAddr': '경기도 수원시 영통구 이의동 0 경기도청',
  'siNm': '경기도',
  'roadAddrPart1': '경기도 수원시 영통구 도청로 30',
  'bdNm': '경기도청',
  'admCd': '4111710300',
  'udrtYn': '0',
  'lnbrMnnm': '0',
  'roadAddr': '경기도 수원시 영통구 도청로 30 (이의동)',
  'lnbrSlno': '0',
  'buldMnnm': '30',
  'bdKdcd': '0',
  'liNm': '',
  'rnMgtSn': '411173177029',
  'mtYn': '0',
  'bdMgtSn': '4111710300100000000000001',
  'buldSlno': '0'},
 {'detBdNmList': '',
  'engAddr': '28 Docheong-ro, Yeongtong-gu, Suwon-si, Gyeonggi-do',
  'rn': '도청로',
  'emdNm': '이의동',
  'zipNo': '16508',
  'roadAddrPart2': ' (이의동)',
  'emdNo': '01',
  'sggNm': '수원시 영통구',
  'jibunAddr': '경기도 수원시 영통구 이의동 0 경기도교육청',
  'siNm': '경기도',
  'roadAddrPart1': '경기도 수원시 영통구 도청로 28',
  'bdNm': '경기도교육청',
  'admCd': '4111710300',
  'udrtYn': '0',
  'lnbrMnnm': '0',
  

In [15]:
type(res['results']['juso'])

list

In [18]:
res['results']['juso'][0]

{'detBdNmList': '',
 'engAddr': '30 Docheong-ro, Yeongtong-gu, Suwon-si, Gyeonggi-do',
 'rn': '도청로',
 'emdNm': '이의동',
 'zipNo': '16508',
 'roadAddrPart2': ' (이의동)',
 'emdNo': '01',
 'sggNm': '수원시 영통구',
 'jibunAddr': '경기도 수원시 영통구 이의동 0 경기도청',
 'siNm': '경기도',
 'roadAddrPart1': '경기도 수원시 영통구 도청로 30',
 'bdNm': '경기도청',
 'admCd': '4111710300',
 'udrtYn': '0',
 'lnbrMnnm': '0',
 'roadAddr': '경기도 수원시 영통구 도청로 30 (이의동)',
 'lnbrSlno': '0',
 'buldMnnm': '30',
 'bdKdcd': '0',
 'liNm': '',
 'rnMgtSn': '411173177029',
 'mtYn': '0',
 'bdMgtSn': '4111710300100000000000001',
 'buldSlno': '0'}

In [19]:
# 원하는 결과 roadAddr
res['results']['juso'][0]['roadAddr']

'경기도 수원시 영통구 도청로 30 (이의동)'

#### 경기도청,장안구청,권선구청,팔달구청,영통구청

In [20]:
# 도로명주소 가져오기 응용
gov_list = '경기도청,장안구청,권선구청,팔달구청,영통구청'.split(',')
gov_list

['경기도청', '장안구청', '권선구청', '팔달구청', '영통구청']

In [22]:
road_addr_list = []
for gov in gov_list:
    params2 = f"keyword={quote(gov)}&resultType=json" 
    url = f'{base_url}?{params1}&{params2}'
    result = requests.get(url)
    if result.status_code == 200:
        res = json.loads(result.text[1:-1])
        road_addr_list.append(res['results']['juso'][0]['roadAddr'])
    else:
        print(result.status_code)

In [23]:
road_addr_list

['경기도 수원시 영통구 도청로 30 (이의동)',
 '경기도 수원시 장안구 송원로 101 (조원동)',
 '경기도 수원시 권선구 호매실로 12 (탑동)',
 '경기도 수원시 팔달구 창룡대로 23 (매향동)',
 '경기도 수원시 영통구 효원로 407 (매탄동)']

In [24]:
import pandas as pd

df = pd.DataFrame({
    '이름': gov_list, '주소': road_addr_list
})
df

,이름,주소
0,경기도청,경기도 수원시 영통구 도청로 30 (이의동)
1,장안구청,경기도 수원시 장안구 송원로 101 (조원동)
2,권선구청,경기도 수원시 권선구 호매실로 12 (탑동)
3,팔달구청,경기도 수원시 팔달구 창룡대로 23 (매향동)
4,영통구청,경기도 수원시 영통구 효원로 407 (매탄동)


In [25]:
df.to_csv('data/수원소재관공서.csv',index=False) # 파일저장
pd.read_csv('data/수원소재관공서.csv') # 파일읽어오기

,이름,주소
0,경기도청,경기도 수원시 영통구 도청로 30 (이의동)
1,장안구청,경기도 수원시 장안구 송원로 101 (조원동)
2,권선구청,경기도 수원시 권선구 호매실로 12 (탑동)
3,팔달구청,경기도 수원시 팔달구 창룡대로 23 (매향동)
4,영통구청,경기도 수원시 영통구 효원로 407 (매탄동)
